In [1]:
import pandas as pd
import numpy as np

In [4]:
df_categorias = pd.read_csv("../datasets/categorias.csv")
df_produtos = pd.read_csv("../datasets/produtos.csv")
df_vendas = pd.read_csv("../datasets/vendas.csv")

In [11]:
df_produtos_categorias = pd.merge(df_categorias, df_produtos, on="id_categoria")

In [25]:
media_estoque = df_produtos_categorias['estoque'].mean()

produtos_abaixo_da_media = df_produtos_categorias.loc[df_produtos_categorias['estoque'] < media_estoque]#condição dentro do loc

produtos_abaixo_da_media.head()


,id_categoria,nome_categoria,id,nome,preco,estoque
0,1,Eletrônicos,1,Notebook Dell,4500,12
5,3,Imagem e Som,4,"Monitor 24""",890,15
6,3,Imagem e Som,9,Webcam Full HD,310,14
7,4,Móveis,5,Cadeira Gamer,1250,9
9,5,Armazenamento,8,HD 1TB,280,19


In [13]:
preco_medio_categoria = df_produtos_categorias.groupby('nome_categoria')['preco'].mean()

In [16]:
df_produtos_categorias_vendas = pd.merge(df_produtos_categorias, df_vendas, left_on="id", right_on="id_produto")


In [17]:
df_produtos_categorias_vendas['valor_total'] = df_produtos_categorias_vendas['preco'] * df_produtos_categorias_vendas['quantidade']

In [24]:
df_vendas_por_categoria = pd.pivot_table(
    df_produtos_categorias_vendas,
    index=["nome_categoria"],
    values=["valor_total"],
    aggfunc='sum'
)

df_vendas_por_categoria.head()

,valor_total
nome_categoria,
Armazenamento,1720
Eletrônicos,25000
Imagem e Som,1510
Móveis,2500
Periféricos,3040


In [30]:
df_produtos_categorias_estoque_risco = df_produtos_categorias.groupby('nome_categoria')['preco'].mean()
df_produtos_categorias_estoque_risco.head()

nome_categoria
Armazenamento     320.000000
Eletrônicos      3400.000000
Imagem e Som      600.000000
Móveis           1250.000000
Periféricos       226.666667
Name: preco, dtype: float64

In [31]:
df_filtro_cat_prod = df_produtos_categorias.groupby('nome_categoria').agg(
    preco_medio = ('preco', 'mean'),
    estoque_total = ('estoque', 'sum')
).reset_index()
df_filtro_cat_prod.head()

,nome_categoria,preco_medio,estoque_total
0,Armazenamento,320.000000,41
1,Eletrônicos,3400.000000,37
2,Imagem e Som,600.000000,29
3,Móveis,1250.000000,9
4,Periféricos,226.666667,101


In [35]:
df_filtrado_quant = df_filtro_cat_prod[
    (df_filtro_cat_prod['preco_medio'] > 20) & #entre parenteses porque é uma "operação"
    (df_filtro_cat_prod['estoque_total'] < 200)
]
df_filtrado_quant = df_filtrado_quant.round(2)

df_filtrado_quant.head()

,nome_categoria,preco_medio,estoque_total
0,Armazenamento,320.00,41
1,Eletrônicos,3400.00,37
2,Imagem e Som,600.00,29
3,Móveis,1250.00,9
4,Periféricos,226.67,101


In [36]:
df_filtrado_quant.to_csv('../datasets/relatorio_critico.csv',index=False, encoding='utf-8')